10)Final Version for 3*3 with 100 percent accuracy. Just with values of cell

In [1]:

from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
import os
current_directory = os.getcwd()
print(current_directory)
dir_base = os.path.join(current_directory,'3_3_small.csv')
print(dir_base)
#des_base = os.path.join(dir_base,'3_3_small.csv')
#print(des_base)

/home/ladans/Learning systems/Sample3_3
/home/ladans/Learning systems/Sample3_3/3_3_small.csv


In [3]:

# Load data
data = pd.read_csv(dir_base)

# Define a function to convert the board into numeric values (1 for 'X', 2 for 'O', 0 for space)
def convert_board_to_numeric(board):
    return [1 if ch == 'X' else 2 if ch == 'O' else 0 for ch in board]

# Apply the conversion and reshape each row into a 3x3 matrix
data['numeric_board'] = data['board'].apply(convert_board_to_numeric)
data['numeric_board_matrix'] = data['numeric_board'].apply(lambda x: np.array(x).reshape(3, 3))

# Split into training and test sets
board_size = 3
subset_size = int(data.shape[0] * 0.9)
X_train = data['numeric_board_matrix'].iloc[:subset_size]
X_test = data['numeric_board_matrix'].iloc[subset_size:]
Y_train = data['winner'].iloc[:subset_size]
Y_test = data['winner'].iloc[subset_size:]

# Convert to list
X_train_list = np.array(X_train.tolist())
X_test_list = np.array(X_test.tolist())
print('f')

f


In [4]:
# Convert Y_train from Pandas Series to NumPy array
Y_train_array = Y_train.to_numpy()

# Similarly, convert Y_test if needed
Y_test_array = Y_test.to_numpy()
Y_train_array

array([1, 0, 0, ..., 1, 1, 0])

In [5]:
X_train[0]

array([[0, 1, 0],
       [1, 1, 2],
       [2, 2, 0]])

In [6]:
# Initialize a dictionary to store the number of edges for each node
node_edges = {}
dim=3
for row in range(dim):
    for col in range(dim):
        node_id = row * dim + col  # Unique node ID for each hexagon

        # Dynamically calculate the number of neighbors for each node
        number_of_neighbors = 0
        if row > 0: number_of_neighbors += 1  # Above
        if row < dim - 1: number_of_neighbors += 1  # Below
        if col > 0: number_of_neighbors += 1  # Left
        if col < dim - 1: number_of_neighbors += 1  # Right
        if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
        if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft
        
        # Store the number of edges for this node
        node_edges[node_id] = number_of_neighbors

# Extract just the values (number of edges) into a list
n_edges_list = list(node_edges.values())

# Output the list of edge values (optional)
print(n_edges_list)

[2, 4, 3, 4, 6, 4, 3, 4, 2]


In [13]:
#this part is responsible for generating edges (connections) between adjacent cells in a 3x3 Hex grid. Rows, columns and Diagonals
import itertools
dim=3
# Function to generate edges for a 3x3 grid
def generate_edges():
    edges = []

    # Row connections (horizontal)
    for row in range(dim):
        for col in range(dim-1):  # 2 because we connect (col, col+1)
            edges.append(((row, col), (row, col+1)))
    
    # Column connections (vertical)
    for col in range(dim):
        for row in range(dim-1):  # 2 because we connect (row, row+1)
            edges.append(((row, col), (row+1, col)))

    # Diagonal connections (from bottom-left to top-right)
    for row in range(dim-1):
        for col in range(dim-1):  # Diagonals will only connect within the bounds
            edges.append(((row, col+1), (row+1, col)))

    return edges

# Generate edges
edges = generate_edges()

edges

[((0, 0), (0, 1)),
 ((0, 1), (0, 2)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 2)),
 ((2, 0), (2, 1)),
 ((2, 1), (2, 2)),
 ((0, 0), (1, 0)),
 ((1, 0), (2, 0)),
 ((0, 1), (1, 1)),
 ((1, 1), (2, 1)),
 ((0, 2), (1, 2)),
 ((1, 2), (2, 2)),
 ((0, 1), (1, 0)),
 ((0, 2), (1, 1)),
 ((1, 1), (2, 0)),
 ((1, 2), (2, 1))]

In [8]:
symbol_names = ["Blue", "Red", "White"]
symbol_names

['Blue', 'Red', 'White']

In [9]:
import argparse
import sys

def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=100, type=int)
    parser.add_argument("--number-of-clauses", default=200, type=int)
    parser.add_argument("--T", default=400, type=int)
    parser.add_argument("--s", default=1.2, type=float)
    parser.add_argument("--depth", default=3, type=int)
    parser.add_argument("--hypervector-size", default=512, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=512, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument('--double-hashing', dest='double_hashing', default=False, action='store_true')
    parser.add_argument("--max-included-literals", default=16, type=int)

    # Add this line to avoid Jupyter arguments causing an error
    args = parser.parse_args(args=[])  # Ignore any command line arguments

    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args
args = default_args()

In [10]:
# Initialize the graph structure for training
graphs_train = Graphs(
    number_of_graphs=subset_size,
    symbols=symbol_names,
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
    double_hashing=args.double_hashing,
)
# Prepare nodes
for graph_id in range(X_train_list.shape[0]):
    graphs_train.set_number_of_graph_nodes(
        graph_id=graph_id,
        number_of_graph_nodes=dim**2,
    )
graphs_train.prepare_node_configuration()

# Prepare edges
for graph_id in range(X_train_list.shape[0]):
    for k in range(dim**2):
        graphs_train.add_graph_node(graph_id, k, n_edges_list[k])
graphs_train.prepare_edge_configuration()


# Add nodes and their properties for each graph
for graph_id in range(X_train_list.shape[0]):
    for k in range(9):  # 9 nodes for the 3x3 grid
        # Get the value of the current cell in the board
        cell_value = X_train_list[graph_id].flatten()[k]  # Flatten to 1D array

        # Assign node properties based on the board state
        if cell_value == 1:  # Player 1
            graphs_train.add_graph_node_property(graph_id, k, "Blue")
        elif cell_value == 2:  # Player 2
            graphs_train.add_graph_node_property(graph_id, k, "Red")
        else:  # Empty cell
            graphs_train.add_graph_node_property(graph_id, k, "White")

# Prepare edge configuration for the training data
graphs_train.prepare_edge_configuration()

# Add edge connections based on the predefined edges
for graph_id in range(X_train_list.shape[0]):
    for edge in edges:
        node_id = edge[0][0] * dim + edge[0][1]  # Convert (row, col) to node ID
        destination_node_id = edge[1][0] * dim + edge[1][1]

        # Add bidirectional edges
        graphs_train.add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type_name=0)
        graphs_train.add_graph_node_edge(graph_id, destination_node_id, node_id, edge_type_name=0)

# Encode the graph
graphs_train.encode()
print('end')

end


In [11]:

# Initialize the graph structure for test data
graphs_test = Graphs(X_test_list.shape[0], init_with=graphs_train)


# Prepare nodes
for graph_id in range(X_test_list.shape[0]):
    graphs_test.set_number_of_graph_nodes(
        graph_id=graph_id,
        number_of_graph_nodes=dim**2,
    )
graphs_test.prepare_node_configuration()

# Prepare edges
for graph_id in range(X_test_list.shape[0]):
    for k in range(dim**2):
        graphs_test.add_graph_node(graph_id, k, n_edges_list[k])
graphs_test.prepare_edge_configuration()


# Add nodes and their properties for each graph
for graph_id in range(X_test_list.shape[0]):
    for k in range(9):  # 9 nodes for the 3x3 grid
        # Get the value of the current cell in the board
        cell_value = X_test_list[graph_id].flatten()[k]  # Flatten to 1D array

        # Assign node properties based on the board state
        if cell_value == 1:  # Player 1
            graphs_test.add_graph_node_property(graph_id, k, "Blue")
        elif cell_value == 2:  # Player 2
            graphs_test.add_graph_node_property(graph_id, k, "Red")
        else:  # Empty cell
            graphs_test.add_graph_node_property(graph_id, k, "White")


# Add edge connections based on the predefined edges
for graph_id in range(X_test_list.shape[0]):
    for edge in edges:
        node_id = edge[0][0] * dim + edge[0][1]  # Convert (row, col) to node ID
        destination_node_id = edge[1][0] * dim + edge[1][1]

        # Add bidirectional edges
        graphs_test.add_graph_node_edge(graph_id, node_id, destination_node_id, edge_type_name=0)
        graphs_test.add_graph_node_edge(graph_id, destination_node_id, node_id, edge_type_name=0)
graphs_test.encode()
print('end')


end


In [12]:
# Train the Tsetlin Machine
tm = MultiClassGraphTsetlinMachine(
    args.number_of_clauses,
    args.T,
    args.s,
    depth=args.depth,
    message_size=args.message_size,
    message_bits=args.message_bits,
    max_included_literals=args.max_included_literals,
    grid=(16*13,1,1),
    block=(128,1,1)
)

start_training = time()
for i in range(args.epochs):
    tm.fit(graphs_train, Y_train_array, epochs=1, incremental=True)
    print(f"Epoch#{i+1} -- Accuracy train: {np.mean(Y_train_array == tm.predict(graphs_train))}", end=' ')
    print(f"-- Accuracy test: {np.mean(Y_test_array == tm.predict(graphs_test))} ")
stop_training = time()
print(f"Time: {stop_training - start_training}")


Initialization of sparse structure.
Epoch#1 -- Accuracy train: 0.8577777777777778 -- Accuracy test: 0.865 
Epoch#2 -- Accuracy train: 0.8216666666666667 -- Accuracy test: 0.835 
Epoch#3 -- Accuracy train: 0.5 -- Accuracy test: 0.48 
Epoch#4 -- Accuracy train: 0.8477777777777777 -- Accuracy test: 0.855 
Epoch#5 -- Accuracy train: 0.8466666666666667 -- Accuracy test: 0.86 
Epoch#6 -- Accuracy train: 0.8833333333333333 -- Accuracy test: 0.85 
Epoch#7 -- Accuracy train: 0.94 -- Accuracy test: 0.955 
Epoch#8 -- Accuracy train: 0.9161111111111111 -- Accuracy test: 0.94 
Epoch#9 -- Accuracy train: 0.8855555555555555 -- Accuracy test: 0.905 
Epoch#10 -- Accuracy train: 0.84 -- Accuracy test: 0.855 
Epoch#11 -- Accuracy train: 0.965 -- Accuracy test: 0.96 
Epoch#12 -- Accuracy train: 0.9761111111111112 -- Accuracy test: 0.965 
Epoch#13 -- Accuracy train: 0.9661111111111111 -- Accuracy test: 0.98 
Epoch#14 -- Accuracy train: 0.8433333333333334 -- Accuracy test: 0.82 
Epoch#15 -- Accuracy train: 